In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### Adecuación de Legacy Chains

##### Cadenas con LLMChain (legacy)
Propósito del ejemplo <br>
Mostrar cómo adecuar el constructor LLMChain a LCEL.

In [ ]:
# Codigo base
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

mixtral = ChatGroq(model="mixtral-8x7b-32768")
# creamos el template
prompt = ChatPromptTemplate.from_template("Hola cómo estás? mi nombre es {name}",)

# Implementacion de LLMChain
from langchain.chains import LLMChain

chain = LLMChain(
    prompt=prompt,
    llm=mixtral)
respuesta1 = chain.invoke(input="Fernando")
respuesta1["text"]

#####  Remplazo con RunnableSequence

In [4]:
from langchain_core.runnables import RunnableSequence

chain = RunnableSequence(first=prompt, last=mixtral )
respuesta3 = chain.invoke("Fernando")
respuesta3.content

'Hola Fernando, ¡mucho gusto en conocerte! Estoy aquí para ayudarte, ¿en qué puedo assistirte hoy?'

##### Remplazo con operador tubo

In [5]:
chain =   prompt |  mixtral
respuesta2 = chain.invoke(input="Fernando")
respuesta2.content

'Hola Fernando, ¡qué tal estás tú? Encantado de saludarte. Si necesitas ayuda con algo, no dudes en preguntar. Estoy aquí para asistirte en lo que necesites. ¿En qué puedo ayudarte hoy?'

#### Cadenas con RetrievalQA
Propósito del ejemplo <br>
RetrievalQA es un constructor de cadenas de uso común que ha sido clasificado como obsoleto. Este ejemplo muestra cómo adecuarlo a una sintaxis LCEL.

In [4]:
from langchain_community.document_loaders import WebBaseLoader, TextLoader
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser

#loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
loader = TextLoader('directorio/LLMPoweredAutonomousAgents.txt')
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)
vectorstore = FAISS.from_documents(documents=all_splits, embedding=HuggingFaceBgeEmbeddings())
llm = ChatGroq(model="mixtral-8x7b-32768")

# Implementacion de RetrievalQA
from langchain import hub
from langchain.chains import RetrievalQA

# Documentacion del prompt en https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")

qa_chain1 = RetrievalQA.from_llm(
    llm, retriever=vectorstore.as_retriever(), prompt=prompt)

respuesta = qa_chain1.invoke("What are autonomous agents?")
print(StrOutputParser().parse(respuesta['result']))
print(type(respuesta))
print("********************************************")

# El remplazo con sintaxis de LCEL
from langchain_core.runnables import (RunnablePassthrough)
retriever = vectorstore.as_retriever()
qa_chain2 = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser()
)

respuesta = qa_chain2.invoke("What are autonomous agents?")
print(respuesta)

Autonomous agents are systems that use a large language model (LLM) as their core controller for problem-solving. These agents consist of several components, including planning and task decomposition. The LLM functions as the agent's brain, guiding its decision-making process. Additionally, autonomous agents often employ self-reflection to learn from past actions, improve decision-making, and rectify errors.
<class 'dict'>
********************************************
Autonomous agents are systems that use a large language model (LLM) as their core controller to perform tasks. They can learn to call external APIs for additional information and improve iteratively through self-reflection, which helps them refine past action decisions and correct previous mistakes.


##### Remplazo con create_stuff_documents_chain y create_retrieval_chain

In [8]:
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Documentacion del prompt en https://smith.langchain.com/hub/langchain-ai/retrieval-qa-chat
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(vectorstore.as_retriever(), combine_docs_chain)

rag_chain.invoke({"input": "What are autonomous agents?"})["answer"]

'Autonomous agents are systems that can make decisions and perform tasks independently without direct human intervention. In the provided context, autonomous agents are powered by a neuro-symbolic architecture called MRKL, which consists of various "expert" modules that can be neural or symbolic. The LLM (likely a language model) acts as a router to direct inquiries to the most suitable expert module. These agents have a planning component that helps them break down complex tasks into smaller steps through a process called task decomposition.'

In [ ]:
from langchain import hub
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# Initialize your retriever (e.g., using Chroma)
vectorstore = Chroma(embedding_function=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

# Initialize your LLM
llm = OpenAI()

# Load a prompt from LangChain Hub
prompt = hub.pull("rlm/rag-prompt")

# Define the LCEL chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser()
)

# Use the chain
response = rag_chain.invoke("What is the capital of France?")
print(response)